In [1]:
!pip install tensorflow==2.3.1 gym keras-rl2 gym[atari]
!pip install tensorflow gym keras-rl2 gym[atari]
!pip install gym[accept-rom-license]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 320.4 MB 22 kB/s 
     |████████████████████████████████| 52 kB 856 kB/s 
     |████████████████████████████████| 20.1 MB 1.2 MB/s 
     |████████████████████████████████| 2.9 MB 35.8 MB/s 
     |████████████████████████████████| 459 kB 41.4 MB/s 
  Attempting uninstall: numpy
    Found existing installation: numpy 1.21.6
    Uninstalling numpy-1.21.6:
      Successfully uninstalled numpy-1.21.6
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.8.0
    Uninstalling tensorflow-estimator-2.8.0:
      Successfully uninstalled tensorflow-estimator-2.8.0
  Attempting uninstall: h5py
    Found existing installation: h5py 3.1.0
    Uninstalling h5py-3.1.0:
      Successfully uninstalled h5py-3.1.0
  Attempting uninstall: gast
    Found existing installation: gast 0.5.3
    Uninstalling gast-0.5.3:
      Su

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
%pip install -U gym>=0.21.0
%pip install -U gym[atari,accept-rom-license]


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.6 MB 8.7 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for AutoROM.accept-rom-license: filename=AutoROM.accept_rom_license-0.4.2-py3-none-any.whl size=441027 sha256=c2fb5f0554cc04dcd90cab8b34d50b195ef971ae274a9d258882bee8288ea95c
  Stored in directory: /root/.cache/pip/wheels/87/67/2e/6147e7912fe37f5408b80d07527dab807c1d25f5c403a9538a
Successfully built AutoROM.accept-rom-license


In [4]:
!pip install colabgymrender

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for colabgymrender: filename=colabgymrender-1.1.0-py3-none-any.whl size=3132 sha256=94043f0e56d16aae36afffc5f5d9046f5b4f0d596c432652af12ddd707a039b9
  Stored in directory: /root/.cache/pip/wheels/f1/0a/2a/86955ea711b461ab7918236fed2568733f75ed677d0524b56c
Successfully built colabgymrender


In [5]:
import gym 
import random
from colabgymrender.recorder import Recorder

Imageio: 'ffmpeg-linux64-v3.3.1' was not found on your computer; downloading it now.
Try 1. Download from https://github.com/imageio/imageio-binaries/raw/master/ffmpeg/ffmpeg-linux64-v3.3.1 (43.8 MB)
Downloading: 8192/45929032 bytes (0.0%)3276800/45929032 bytes (7.1%)7069696/45929032 bytes (15.4%)10821632/45929032 bytes (23.6%)14491648/45929032 bytes (31.6%)18137088/45929032 bytes (39.5%)21512192/45929032 bytes (46.8%)23928832/45929032 bytes (52.1%)26828800/45929032 bytes (58.4%)29302784/45929032 bytes (63.8%)31653888/45929032 bytes (68.9%)33808384/45929032 bytes (73.6%)35651584/45929032 bytes (77.6%)

In [6]:
env = gym.make('SpaceInvaders-v0')
env = Recorder(env, './video')
height, width, channels = env.observation_space.shape
actions = env.action_space.n


In [7]:
env.unwrapped.get_action_meanings()

['NOOP', 'FIRE', 'RIGHT', 'LEFT', 'RIGHTFIRE', 'LEFTFIRE']

In [8]:
episodes = 5
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
    
    while not done:
        env.render(mode='rgb_array')
        action = random.choice([0,1,2,3,4,5])
        n_state, reward, done, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))
env.close()

Episode:1 Score:135.0
Episode:2 Score:50.0
Episode:3 Score:150.0
Episode:4 Score:220.0
Episode:5 Score:135.0


In [9]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Convolution2D
from tensorflow.keras.optimizers import Adam

In [10]:
def build_model(height, width, channels, actions):
    model = Sequential()
    model.add(Convolution2D(32, (8,8), strides=(4,4), activation='relu', input_shape=(3,height, width, channels)))
    model.add(Convolution2D(64, (4,4), strides=(2,2), activation='relu'))
    model.add(Convolution2D(64, (3,3), activation='relu'))
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dense(256, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    return model

In [16]:
model = build_model(height, width, channels, actions)

In [17]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 3, 51, 39, 32)     6176      
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 3, 24, 18, 64)     32832     
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 3, 22, 16, 64)     36928     
_________________________________________________________________
flatten_1 (Flatten)          (None, 67584)             0         
_________________________________________________________________
dense_4 (Dense)              (None, 512)               34603520  
_________________________________________________________________
dense_5 (Dense)              (None, 256)               131328    
_________________________________________________________________
dense_6 (Dense)              (None, 6)                

In [18]:
from rl.agents import DQNAgent
from rl.memory import SequentialMemory
from rl.policy import LinearAnnealedPolicy, EpsGreedyQPolicy

In [19]:
def build_agent(model, actions):
    policy = LinearAnnealedPolicy(EpsGreedyQPolicy(), attr='eps', value_max=1., value_min=.1, value_test=.2, nb_steps=500)
    memory = SequentialMemory(limit=1000, window_length=3)
    DeepQN = DQNAgent(model=model, memory=memory, policy=policy,enable_dueling_network=True, dueling_type='avg', nb_actions=actions, nb_steps_warmup=1000)
    return DeepQN

In [20]:
DeepQN = build_agent(model, actions)
DeepQN.compile(Adam(lr=1e-4))

In [21]:
DeepQN.fit(env, nb_steps=500, visualize=False, verbose=2)

Training for 500 steps ...
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
done, took 24.818 seconds


In [22]:
scores = DeepQN.test(env, nb_episodes=10, visualize=False)
print(np.mean(scores.history['episode_reward']))
env.play()

Testing for 10 episodes ...
Episode 1: reward: 25.000, steps: 920
Episode 2: reward: 0.000, steps: 922
Episode 3: reward: 0.000, steps: 923
Episode 4: reward: 0.000, steps: 915
Episode 5: reward: 0.000, steps: 918
Episode 6: reward: 0.000, steps: 923
Episode 7: reward: 0.000, steps: 928
Episode 8: reward: 0.000, steps: 930
Episode 9: reward: 0.000, steps: 920
Episode 10: reward: 0.000, steps: 923
2.5
